<a href="https://colab.research.google.com/github/GitEmmSt/Medical_Imaging/blob/main/Practical_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Medical Imaging/practical_session_1

/content/drive/MyDrive/Medical Imaging/practical_session_1


# <center>Biomedical Imaging</center><br><center>Practical session 1: Image formation and reconstruction  (12/10/2021)</center>
***
*Charlotte Thyssen, Jens Maebe, Stefaan Vandenberghe* <br>
*MEDISIP, Ghent University* <br>

<font color=blue>Insert students names and IDs here</font>

***
# General
In total, there will be 4 practical sessions. For every session you should hand in a report **in groups of 2**. These reports will count for 5 out of 20 points for the exam. The topics of the sessions:

1. Image reconstruction in tomography, CT (25%)	

2. Monte Carlo simulations, SPECT (25%)	

3. Image processing part I (25%)

4. Image processing part II (25%)

***
Report due 2 weeks after session


- **Concise** and **to the point** answers to the questions

- Show that you understood what was going on and what the purpose of the exercise was

- Illustrate with figures 

- Hand in your notebook (also add the html file) via UFORA (UGent) or Canvas (VUB)

***
We are here to help you: ask questions!
- During the sessions
- Via mail:
    - cathysse.thyssen@ugent.be
    - jens.maebe@ugent.be
    

**Strict deadline for this session: 26 October 2021 at 23:59**.

During the sessions, we will use **scikit-image**. This is a free python toolbox with a collection of algorithms for image processing. In image processing, all images are nd-arrays (2d if black/white flat image), so in order to be able to load these images. You should always import **numpy**.

# 1. Image formation
## 1.1 Fanbeam transform
In a CT scanner, the photon source is an X-ray tube. To make things easier, we approximate it as an infinitesimally small point source (figure 1).

<table>
    <tr>
        <td><img src="images/CT_schematic.jpg" width="300"/><figcaption><center>(a)</center></figcaption></td>
        <td><img src="images/CT_open.jpg" width="300"/><figcaption><center>(b)</center></figcaption></td>
    </tr>
</table>

<center><i>Figure 1: (a) The different parts of a CT-scanner. (b) The inside of a CT scanner with the different parts and coordinates indicated.</i></center>

From this point source, photons are fired through the patient's body. The curved detector on the other side of the patient will detect the photons which have not been attenuated along their path through the body. Because we knew the number of fired photons ($I_0$), we can calculate the attenuation from the Beer-Lambert law:
\begin{equation}
	I=\int I_0\left(E\right) e^{-\int_0^L\mu(\vec{x},E) d\vec{x}} dE,
\end{equation}

with $I$ the number of detected photons (quanta) in a detector pixel, $E$ the photon energy (remember, X-rays generated by an X-ray tube are always polyenergetic!), $\mu$ the attenuation coefficient and $\vec{x}$ a vector representing the spatial coordinate along the line, from $0$ to $L$. If we ignore the energy spectrum for now and try to determine $\int \mu(\vec{x}) d\vec{x}$, we first have to calculate the log-attenuated values:

\begin{equation}
\int_0^L \mu(\vec{x}) d\vec{x} = \log\left(I_0\right) - \log\left( I \right).
\end{equation} 

<table>
    <tr>
        <td><img src="images/fanbeam.jpg" width="300"/><figcaption><center>(a)</center></figcaption></td>
        <td><img src="images/parbeam.jpg" width="300"/><figcaption><center>(b)</center></figcaption></td>
    </tr>
        <td><img src="images/fansino.jpg" width="300"/><figcaption><center>(c)</center></figcaption></td>
        <td><img src="images/parsino.jpg" width="300"/><figcaption><center>(d)</center></figcaption></td>
    <tr>
</table>
<center><i>Figure 2: (a) The geometry of a fanbeam projection, (b) parallel beam projection, (c) a sinogram obtained from a fanbeam, (d) and parallel beam projection.
</i></center>

These so-called path lengths are what will be used for the rest of the practicum session.

The ideal path followed by each photon can be represented by a line connecting the X-ray source with the place of detection. All lines (for all detector pixels) for one rotation angle together form one fan beam projection (figure 2(a)). This represents one line in the sinogram. If we now rotate the source-detector pair around the patient and sample on different angles, a complete sinogram is obtained. 

This measured sinogram, containing data acquired from different angles, is the only data received from a CT scanner. This data can be used to directly reconstruct the original object in 3D. However, in this exercise lesson, we will reconstruct only from parallel projections and not from fan beam projections. Therefore, the fan beam projections have to be converted (a.k.a. rebinned) to parallel projection data.

## 1.2 The radon transform
Figure 2(c) and 2(d) respectively show sinograms obtained in fanbeam geometry and a parallel beam geometry. Forward projecting in a parallel beam geometry can be described mathematically by the Radon transform $R(\theta,x^\prime)$: 
\begin{equation}
	R(\theta,x^\prime)= \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} f(x,y)\delta(x^\prime - (x\cos\theta+y\sin\theta)) dx dy.
\end{equation}
Here, $f(x,y)$ represents the distribution of the attenuation coefficients in the body.
The Radon transform $R(\theta=\Theta,x\prime=X^\prime)$ can be interpreted as the sum (if we discretize $f(x,y)$ in pixels) of the attenuation coefficients at rotation angle $\Theta$ along a line perpendicular to the $x^\prime$-axis, crossing this axis at $X^\prime$. $x^\prime$ can be calculated from $\theta$, $x$ and $y$ as:
\begin{equation}
x^\prime= x \cos\theta +y \sin\theta,
\end{equation}
which is just a rotated $x$. These data can then be reconstructed by using the inverse of the Radon transform.

## <font color='blue'>Exercise 1: Parallel beam projection</font>
In this exercise you will create a virtual CT scanner data. However, unlike in reality, we will assume a perfect parallel beam which allows us to use the Radon transform for this projection. For this exercise and the following, we will use the scikit-image package of python (https://scikit-image.org/).   

Start by creating an empty 64$\times$64 image (2D matrix) where you add a point (cluster of 2$\times$2 pixels with value 1) somewhere off-center. Project this data over a 360$^\circ$ angular range using 360 projection angles by making use of the `radon(...)` function from scikit-image.

Repeat this with a 6 pixel long off-center 'line' instead of a point.


### Remark
Define the projection angles with the `np.linespace(...)` function from the numpy package. This will generate an array with evenly distributed angles over a given range.

<font color='blue'>We expect to see the two images you used with their sinograms. Explain the differences you see between both sinograms and reinforce your answer by making use of overlapping profiles (line plots) of the sinograms at certain angles. </font>

<font color='red'> SOLUTION: </font>

YOUR ANWER TO THE QUESTION

In [ ]:
### Code for Exercise 1

***
## <font color='blue'>Exercise 2: the sinogram</font>
To understand the content of a sinogram, we will take a closer look at the relation between an image and its sinogram. Therefore, start by making an image containing four points (each a cluster of 2$\times$2 pixels). The first point should be in the center, the three other points should be located off-center, at different distances from the center. Make use of the `radon(...)` function in scikit-image to project the image to sinogram space.


<font color='blue'>Plot the original image and show it together with its sinogram obtained from a 360◦ and a 180◦ angular range. Use 90 projection angles in both cases.

Now answer these questions:
- Why do we call this a sinogram?
- What is the relation between the sinogram and the position in the original image?
- Would you use the 360◦ or the 180◦ angular range when the number of projection angles is fixed? Why? </font>

<font color='red'> SOLUTION: </font>

YOUR ANWER TO THE QUESTION

In [ ]:
### Code for Exercise 2

***
## <font color='blue'>Exercise 3: the sinogram (continued)</font>
To get an even better feel of what kind of data is included in a sinogram, download the file **Abdomen.bin** from Ufora and open it. This data is a slice of an in-vivo abdominal scan of a mouse, measured with a cone-beam micro-CT system in the Infinity lab. We already rebinned the cone-beam sinogram to parallel beam geometry for you and below you can find the code to import the sinogram data. Make use of the `iradon(...)` function from scikit.image to reconstruct the sinogram into an image.


<font color='blue'>Answer these questions:

- What is wrong with this image?
- What could be the physical cause (a scanner problem) of this? (Think about all elements contained in a CT system!)
- Correct for this artefact on the sinogram; plot the improved image and explain what you did</font>

In [ ]:
import numpy as np

f=open("Abdomen.bin", 'rb')
data = np.fromfile(f, dtype=np.float64)
sinogram = np.transpose(np.reshape(data, [360,729]))
f.close()

<font color='red'> SOLUTION: </font>

YOUR ANWER TO THE QUESTION

In [ ]:
### Code for Exercise 3

***
## <font color='blue'>Exercise 4: Build your own forward projector</font>
In medical image reconstruction, the name ‘projector’ or ‘forward projector’ is given to an operation
which transforms data from image space to projection space (the sinogram). In this exercise, we want
you to implement your own projector in Python, as an alternative to the `radon(...)` function. Implement this projector by combining image rotation followed by summation along one axis. You should keep x' and sum along y'.

Start with a 256×256 image that you generate with the `shepp_logan_phantom(...)` function from skimage.data. Now use the functions `skimage.transform.rotate(...)` and `np.sum(...)` to perform projections and to build a sinogram. Use 180 projection angles over an angular range of 180 . Also use `radon(...)` to forward project the original image, so we can compare your implementation to the one you try to mimic.


<font color='blue'>Show the sinogram obtained with your own projector. </font>

In [ ]:
from skimage.data import shepp_logan_phantom
from skimage.transform import rescale

image = shepp_logan_phantom()
image = rescale(image, scale=256/400, mode='reflect', multichannel=False)

<font color='red'> SOLUTION: </font>

YOUR ANWER TO THE QUESTION

In [ ]:
### Code for Exercise 4

## 2. Image reconstruction
### 2.1 Backprojection
The backprojection operation describes the transformation back from projection space to image space and it is the mathematical adjoint of the Radon transform:

\begin{equation}
B(x,y) = 
\int_0^\pi R(\theta,x')d\theta
\end{equation} 

Here, $B(x, y)$ represents the backprojected image, and $R(\theta, x′)$ is the sinogram. For one fixed angle $\Theta$, the backprojection $B(x, y; \theta = \Theta)$ is performed by smearing the values of $R(\theta = \Theta, x′)$ along perpendicular lines (Figure 4). If we repeat this operation for all angles $\theta$, the sum of all $B(x,y;\theta = \Theta)$ will form $B(x, y)$.

<table>
    <tr>
        <td><img src="images/backproj.jpg" width="300"/><figcaption><center>(a)</center></figcaption></td>
    </tr>
</table>
<center><i>Figure 4: Backprojection of all values of R(θ,x′) to a rotated image for a fixed angle Θ.
</i></center>


***
## <font color='blue'>Exercise 5: Build your own backprojector</font>
We can backproject the sinogram obtained in the previous exercise by first smearing $R(\theta = \theta,x′)$ into a rotated image. When we rotate this image back to the original position and sum over all angles, we obtain a backprojection of the sinogram. Again, use the functions `imrotate(...)` and `sum(...)` to implement a backprojector. Work on the sinogram you created in exercise 2. 

<font color='blue'>Show following images:
- your backprojection for 10 angles over a range of 360◦
- your backprojection for 90 angles over a range of 360◦

Answer these questions:
- What is the difference between the original image and your backprojection image? 
- How can the backprojection be improved? </font>

<font color='red'> SOLUTION: </font>

YOUR ANWER TO THE QUESTION

In [ ]:
### Code for Exercise 5

## 2.3 Filtered backprojection
To understand the origin of the blur in the reconstructed image, take a look at the sampling in the Fourier domain.
Take the backprojection from a single angle:

\begin{equation}
B(x,y;\theta=\Theta) = R(\theta = \Theta,x')
\end{equation} 

$$\begin{eqnarray}
\mathcal{B}(\nu_x,\nu_y;\theta=\Theta) &=& \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} B(x,y;\theta=\Theta) e^{-2\pi i(x\nu_x+y\nu_y)}dx dy \\
&=&  \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} R(\theta=\Theta,x^\prime) e^{-2\pi i(x\nu_x+y\nu_y)}dx^\prime dy^\prime \\
&=& \mathcal{R}(\theta=\Theta,\nu_{x^\prime}) \delta(\nu_{y^\prime}) \\
\end{eqnarray} $$

From the central slice theorem (course notes) we know that $\mathcal{R}(\theta=\Theta,\nu_{x^\prime})=\mathcal{F}(\nu_x,\nu_y)\mid_{\nu_{y^\prime}=0}$. Thus, **every backprojection along 1 angle is actually equal to one line of the Fourier transform of the original image!** Superimposing all lines and then taking the inverse Fourier transform would result in our blurry backprojection image. In other words, the blur in the reconstructed image arises from the fact that the center of the Fourier space will be oversampled due to the radial sampling. You get an overestimation of the low frequency components and an underestimation of the high frequency components. This is illustrated in figure 5.

<table>
    <tr>
        <td><img src="images/sampling_fourier.jpg" width="300"/><figcaption><center>(a)</center></figcaption></td>
        <td><img src="images/grid.jpg" width="300"/><figcaption><center>(b)</center></figcaption></td>
        <td><img src="images/ramp.jpg" width="300"/><figcaption><center>(c)</center></figcaption></td>
</table>
<center><i>Figure 5: (a) Sampling of the Fourier space using 20 backprojections. (b) The radial grid on which the Fourier space is sampled. (c) The ramp filtered used to compensate for the oversampling.
</i></center>

To compensate for the oversampling at lower frequencies, you should use a filter which is proportional to the circumference of a circle defined with radius $\nu =\sqrt{\nu_x^2+\nu_u^2}$. Proportionality with the circumference is equal to proportionality with the radius. Thus our filter will be $\nu$, also called a ramp filter (Figure 5(c)). Thanks to the above, we can discriminate three ways to get to our filtered backprojection, of which the second is the simplest and will be used in the next exercise:

1. **Projections** $\rightarrow$ backprojection $\rightarrow$ 2D FT $\rightarrow$ filter with 2D ramp filter $\rightarrow$ 2D IFT $\rightarrow$ **Filtered reconstructed image**
2. **Projections** $\rightarrow$ 1D FT $\rightarrow$ filter with 1D ramp filter $\rightarrow$ 1D IFT $\rightarrow$ backprojection $\rightarrow$ **Filtered reconstructed image**
3. **Projections** $\rightarrow$ backprojection $\rightarrow$ superimposing all lines in correct angle $\rightarrow$ interpolation to get more uniform grid in 2D FT space $\rightarrow$ 2D IFT $\rightarrow$ **Filtered reconstructed image**


***
## <font color='blue'>Exercise 6: Filtered backprojection</font>
In this exercise, you will have to solve the problem of oversampling with the second method. Work with the sinogram obtained with radon in exercise 4 (Shepp-Logan phantom). Use all 90 angles. First you will have to transform the sinogram to Fourier space. Next, filter your projection data using a ramp filter. After returning to sinogram space with an inverse Fourier transform, backproject the filtered sinogram using your backprojector from the previous exercise. Compare your result to the result of applying `iradon(...,‘Ramp’)` to your unfiltered sinogram data. Also compare with your own backprojector without filtering.

### Remark
In order to generate a ramp filter, use the given function below.

<font color='blue'> 
Plot an image of both your own filtered and unfiltered reconstructions and the iradon-backprojection. Also show the original and filtered sinograms. Is the result better with filtering? Limit the number of angles to 20. What happens now?</font>

In [ ]:
import math

def rampfilter(len):
    # Returns the Fourier Transform of the filter which will be 
    # used to filter the projections
    # INPUT ARGS:   len    - the length of the projections
    # OUTPUT ARGS:  filt   - the filter to use on the projections
    nextpow2 = math.ceil(math.log2(abs(2*len)))
    order = max(64,pow(2,nextpow2))
    filt = np.arange(0,order+1,2)/order
    w = 2*math.pi*np.arange(filt.shape[0])/order # frequency axis up to Nyquist 
    filt[np.where(w>math.pi)] = 0 # Crop the frequency response
    filt_symm = np.zeros([order,1])
    filt_symm[0:filt.shape[0],0] = filt
    filt_symm[filt.shape[0]:,0] = np.flip(filt)[0:-2]
    return (filt_symm)

<font color='red'> SOLUTION: </font>

YOUR ANWER TO THE QUESTION

In [ ]:
### Code for Exercise 6